In [11]:
import numpy as np
from astropy.io import ascii
import pandas as pd
import matplotlib.pyplot as plt
import pystan as ps
import arviz as av
import seaborn as sns
import pickle

%matplotlib inline

In [4]:
data = ascii.read('/Users/samberek/Courses/1501project/data/table5/NGC1407_GC.cat')

In [5]:
data

ID,RA,DEC,RV,RV_err,RAD
str12,float64,float64,int64,int64,float64
NGC1407_S1,55.031092,-18.499864,1436,12,4.93
NGC1407_S2,55.052983,-18.594181,1883,7,0.87
NGC1407_S3,55.060592,-18.592561,1509,5,0.98
NGC1407_S4,55.074404,-18.552261,2008,6,2.19
NGC1407_S5,55.217613,-18.502664,1966,5,10.63
NGC1407_S6,54.986208,-18.603747,1951,6,3.86
NGC1407_S7,55.05315,-18.607761,2090,8,1.67
NGC1407_S8,55.193054,-18.454867,1572,11,11.1
NGC1407_S9,55.064321,-18.564747,1861,11,1.25


In [8]:
type(data['RA'])

astropy.table.column.Column

Data needs to be in a dictionary to be passed to Stan

In [9]:
#I think these need to be numpy arrays, they should already be but potential problem point
standata = {
    'N': len(data),

    'ra_obs': data['RA'],
    'dec_obs': data['DEC'],
    
    'r_obs': data['RAD'],
    'rv_obs': data['RV'],
    'rv_err': data['RV_err'],
}

Function which generates a dictionary with initial values. From Jeff: initial values must be specified for either all parameters xor none of the parameters. The type for each parameter must match what is defined in the Stan model.

In [10]:
# def init_gen():
#     return {
        
#         #there needs to be a better prior here
#         'rho0': np.random.uniform(0., 10**14),
#         'a': np.random.uniform(0., 20.),
        
#         'z': np.full(standata['N'], 1e-6),
#         'yv': np.full(standata['N'], 1e-6),
#         'zv': np.ones(standata['N']),
#         'xv': np.random.uniform(0, max(standata['r_measured']), size=int(standata['N']))
#     }

In [24]:
with open('single_hernquist.stan', 'r') as f:
    stancode = f.read()
    
sm = ps.StanModel(model_code=stancode)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_38d681e95bb95188d480258c74c2fe63 NOW.
